## Prerequisites

1. Go to https://colab.research.google.com/
2. Upload this notebook with `File` -> `Upload Notebook` and upload this notebook

In [ ]:
!uv pip install -q syft-flwr

## Login

In [ ]:
import syft_client as sc

import syft_flwr

print(f"{sc.__version__ = }")
print(f"{syft_flwr.__version__ = }")

ds_email = input("Enter the Data Scientist's email: ")
ds_client = sc.login_ds(email=ds_email)

## Peer Addition

In [ ]:
do1_email = input("Enter the First Data Owner's email: ")
ds_client.add_peer(do1_email)

In [ ]:
do2_email = input("Enter the Second Data Owner's email: ")
ds_client.add_peer(do2_email)

ds_client.peers

## Explore Datasets

DO1's datasets

In [ ]:
do1_datasets = ds_client.datasets.get_all(datasite=do1_email)
do1_datasets

In [ ]:
do1_datasets[0].describe()

DO2's datasets

In [ ]:
do2_datasets = ds_client.datasets.get_all(datasite=do2_email)
do2_datasets

In [ ]:
do2_datasets[0].describe()

Append the mock paths to the list of mock datasets (needed to run simulation locally)

In [ ]:
mock_dataset_urls = [do1_datasets[0].mock_url, do2_datasets[0].mock_url]
mock_dataset_urls

## Propose FL Project

Clone the project from github

In [ ]:
from pathlib import Path

!mkdir -p /content/fl-diabetes-prediction
!curl -sL https://github.com/khoaguin/fl-diabetes-prediction/archive/refs/heads/main.tar.gz | tar -xz --strip-components=1 -C /content/fl-diabetes-prediction

SYFT_FLWR_PROJECT_PATH = Path("/content/fl-diabetes-prediction")

print(f"syft-flwr project at: {SYFT_FLWR_PROJECT_PATH}")

Bootstrap the `syft-flwr` project

In [ ]:
import syft_flwr

try:
    !rm -rf {SYFT_FLWR_PROJECT_PATH / "main.py"}
    print(f"syft_flwr version = {syft_flwr.__version__}")
    do_emails = [peer.email for peer in ds_client.peers]
    syft_flwr.bootstrap(
        SYFT_FLWR_PROJECT_PATH, aggregator=ds_email, datasites=do_emails
    )
    print("Bootstrapped project successfully ✅")
except Exception as e:
    print(e)

## Submit FL Project to DOs

In [ ]:
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fl_diabetes_prediction" / "__pycache__"}  # clean before submitting

job_name = "fl-diabetes-training"

ds_client.submit_python_job(
    user=do1_email,
    code_path=str(SYFT_FLWR_PROJECT_PATH),
    job_name=job_name,
)

In [ ]:
ds_client.submit_python_job(
    user=do2_email,
    code_path=str(SYFT_FLWR_PROJECT_PATH),
    job_name=job_name,
)

In [ ]:
ds_client.jobs

## Temp Fix: DS installs packages and runs `main.py` directly

In [ ]:
!uv pip install \
    "flwr-datasets>=0.5.0" \
    "imblearn>=0.0" \
    "loguru>=0.7.3" \
    "pandas>=2.3.0" \
    "ipywidgets>=8.1.7" \
    "scikit-learn==1.7.1" \
    "torch>=2.8.0" \
    "ray==2.31.0"

In [ ]:
assert (
    SYFT_FLWR_PROJECT_PATH.exists()
), f"Project path does not exist: {SYFT_FLWR_PROJECT_PATH}"
assert (
    SYFT_FLWR_PROJECT_PATH / "main.py"
).exists(), f"main.py not found in project path: {SYFT_FLWR_PROJECT_PATH / 'main.py'}"

ds_email = ds_client.email
syftbox_folder = f"/content/SyftBox_{ds_email}"
!SYFTBOX_EMAIL="{ds_email}" SYFTBOX_FOLDER="{syftbox_folder}" \
    uv run {str(SYFT_FLWR_PROJECT_PATH / "main.py")}

In [ ]:
ds_client.jobs

## Clean Up

In [ ]:
ds_client.delete_syftbox()